In [1]:
%cd /home/q123/Desktop/explo

import torch 
import gpytorch 
import logging
import logging.config

from src.helpers import setup_experiment
from src.trainer import Trainer
from src.optimizers.gibo import GIBOptimizer
from src.optimizers.vanilla import BOptimizer
from src.config import get_configs

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_name = "CartPole-v1"
kernel_name = "rbfstate" ## "linearstate" /"rbfstate"

env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
additional_layers=[] ### can be empty or [8,7] for adding 2 layers with width 8,7 respectively

kernel_config = {
        "use_ard":False,
        "kernel_name":kernel_name,
        # "lengthscale_hyperprior":gpytorch.priors.torch_priors.GammaPrior(3.0,6.0),
        # "lengthscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.001),
        # "outputscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.01),
        # "outputscale_hyperprior":gpytorch.priors.torch_priors.NormalPrior(loc=2.0,scale=1.0),
        }


optimizer_config = {
        "n_eval":1,
        ### for GIBO
        "n_max":20, 
        "n_info_samples":8,
        "delta":0.1, ## 0.01 better for linear
        ### hessian normalisation applies only for rbf
        "normalize_gradient":True if kernel_name == "rbf" else False,
        "standard_deviation_scaling":False,
}


trainer_config = {
        "n_steps" :200,
        "report_freq":10,
        "save_best":False,
}




model,objective_env = setup_experiment(env_config,kernel_config,likelihood_config,additional_layers)


### Chose optimizer 
#optimizer = BOptimizer(**optimizer_config)
optimizer = GIBOptimizer(model,**optimizer_config)
trainer = Trainer(model,objective_env,optimizer,**trainer_config)
rslt= trainer.run()

MathLog.src.helpers : WARNING : MLP dimensions : [4, 1]


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gpytorch/lazy/lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


Using ard_num_dims = 5
Statekernel i have mlp MLP()
 Gibo will use 20 last points to fit GP and 8 info samples
current 0.09200000017881393 / max 0.284000039100647 /batch_mean 0.11020000278949738 /batch_max 0.164000004529953 
##############################
covar_lengthscale max 7.971510410308838 / min 7.971510410308838                      covar_outputscale 3.3398217169633426e-07                     noise 0.20000067353248596
##############################
last parameters tensor([ 0.0168, -0.5591,  0.1512,  0.8810,  0.0425])
MLL : -0.11691627651453018
current 0.07400000840425491 / max 0.284000039100647 /batch_mean 0.1072000041604042 /batch_max 0.1940000206232071 
##############################
covar_lengthscale max 9.060495376586914 / min 9.060495376586914                      covar_outputscale 2.3877496460045222e-06                     noise 0.2000000774860382
##############################
last parameters tensor([-0.0149, -0.4880,  0.0107,  0.6922,  0.2350])
MLL : -0.12173104286193848


KeyboardInterrupt: 